# loading the Dataset

In [1]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk
from sklearn.feature_extraction.text import CountVectorizer
import collections
from collections import defaultdict
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
data = pd.read_csv('/content/drive/MyDrive/NLP-Python/data/IMDB Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# Data Pre-Processing 

In [3]:
def remove_tags(strings):
  removelist = ""
  result = re.sub('','',strings) # Remove HTML tags
  result = re.sub('https://.*','',result) # Remove URLS
  result = re.sub(r'[^w'+removelist+']','',result) # Remove non-alphanumeric charecters
  result = result.lower()
  return result

data['review'] = data['review'].apply(lambda cw : remove_tags(cw))
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
data['review'] = data['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Lemmatization

lemmatization is used to find the root form of words or lemmas in NLP. This helps save unnecessary computational overhead in trying to decipher entire words.

In [7]:
# !pip install nltk --upgrade
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    st = ""
    for w in w_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st
data['review'] = data.review.apply(lemmatize_text)

# Encoding labels and Making Train-test Splits

In [8]:
reviews = data['review'].values
labels = data['sentiment'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

The Data is then split into 80-20 splits from sklearn.model_selection

In [9]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, stratify = encoded_labels)

# Building the Naive Bayes Classifier 

In [12]:
vec = CountVectorizer(max_features = 3000)
X = vec.fit_transform(train_sentences)
vocab = vec.get_feature_names()
X = X.toarray()
word_counts = {}
for l in range(2):
    word_counts[l] = defaultdict(lambda: 0)
for i in range(X.shape[0]):
    l = train_labels[i]
    for j in range(len(vocab)):
        word_counts[l][vocab[j]] += X[i][j]

Laplace smoothing - Takes the vocabulary and the raw word_counts dictionary and returns the smoothened conditional probabilities

In [13]:
def laplace_smoothing(n_label_items, vocab, word_counts, word, text_label):
  a = word_counts[text_label][word] + 1
  b = n_label_items[text_label] + len(vocab)
  return math.log(a/b)

In [22]:
def group_by_label(x, y, labels):
    data = {}
    for l in labels:
        data[l] = x[np.where(y == l)]
    return data

In [23]:
def fit(x, y, labels):
    n_label_items = {}
    log_label_priors = {}
    n = len(x)
    grouped_data = group_by_label(x, y, labels)
    for l, data in grouped_data.items():
        n_label_items[l] = len(data)
        log_label_priors[l] = math.log(n_label_items[l] / n)
    return n_label_items, log_label_priors

In [24]:
def predict(n_label_items, vocab, word_counts, log_label_priors, labels, x):
    result = []
    for text in x:
        label_scores = {l: log_label_priors[l] for l in labels}
        words = set(w_tokenizer.tokenize(text))
        for word in words:
            if word not in vocab: continue
            for l in labels:
                log_w_given_l = laplace_smoothing(n_label_items, vocab, word_counts, word, l)
                label_scores[l] += log_w_given_l
        result.append(max(label_scores, key=label_scores.get))
    return result

# Fitting the Model on Training SEt and evaluating accuracies on the test set

In [25]:
labels = [0,1]
n_label_items, log_label_priors = fit(train_sentences, train_labels, labels)
pred = predict(n_label_items, vocab, word_counts, log_label_priors, labels, test_sentences)
print("Accuracy of prediction on Test set: ",accuracy_score(test_labels, pred))

Accuracy of prediction on Test set:  0.51832


## Key Takeaways About NB Classifier:
  1. A Naive bayes classifier is a probabilistic ML classifier based on Bayes Theorem.
  2. Laplace Smoothing needs to be performed while calculating feature conditional probabilities so that, unseen words which are not in the trainig corpus can be handled